In [109]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import SMOTE, ADASYN
import torch
from torch import nn
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
STOP = [ 'stop', 'the', 'to', 'and', 'a', 'in', 'it', 'is', 'I', 'that', 'had', 'on', 'for', 'were', 'was']

In [110]:
train_sentences = pd.read_csv('train_with_label.txt', sep = '\t', \
                              names = ['instance_id', 'sentence1', 'sentence2', 'label'], \
                             quoting = 3)

train_df = pd.DataFrame(train_sentences)

In [111]:
def get_data(df):
    sent1 = df.loc[:, 'sentence1']
    sent2 = df.loc[:, 'sentence2']
    labels = df.loc[:, 'label']
    
    return sent1, sent2, labels

In [112]:
train_sent1, train_sent2, train_labels = get_data(train_df)

In [472]:
STOPWORDS = stopwords.words('english')
STOPWORDS = set(STOPWORDS)
# if word not in STOP 
def text_prepare(text, STOPWORDS):
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, '', text)
    text = re.sub(' +', ' ', text)
    #rx =  re.compile('[\n\'()\w]+|\.]')
    #text = re.sub(rx, ' ', text)
    #text = re.sub(' +', ' ', text)
    text = text.lower()

    # delete stopwords from text
    text = ' '.join([word for word in text.split()]) 
    text = text.strip()
    return text

In [473]:
tokall1 = []
tokall2 = []
for i in range(len(train_df)):
    tok1 = ''
    tok2 = ''
    tok1 = text_prepare(train_sent1[i], STOPWORDS)
    tok2 = text_prepare(train_sent2[i], STOPWORDS)
    tokall1.append(tok1)
    tokall2.append(tok2)

In [474]:
dev_sentences = pd.read_csv('dev_with_label.txt', sep = '\t', \
                              names = ['instance_id', 'sentence1', 'sentence2', 'label'], \
                             quoting = 3)

dev_df = pd.DataFrame(dev_sentences)

In [475]:
dev_sent1, dev_sent2, dev_labels = get_data(dev_df)

In [476]:
dev_tokall1 = []
dev_tokall2 = []
for i in range(len(dev_df)):
    dev_tok1 = ''
    dev_tok2 = ''
    dev_tok1 = text_prepare(dev_sent1[i], STOPWORDS)
    dev_tok2 = text_prepare(dev_sent2[i], STOPWORDS)
    dev_tokall1.append(dev_tok1)
    dev_tokall2.append(dev_tok2)

In [505]:
fuzz_similarity = []
for i in range(len(tokall1)):
    ratio = fuzz.token_set_ratio(tokall1[i], tokall2[i])
    fuzz_similarity += [ratio]

In [506]:
dev_fuzz_similarity = []
for i in range(len(dev_tokall1)):
    ratio = fuzz.token_set_ratio(dev_tokall1[i], dev_tokall2[i])
    dev_fuzz_similarity +=[ratio]

In [507]:
for i in range(len(fuzz_similarity)):
    fuzz_similarity[i] = fuzz_similarity[i] / 10

In [508]:
for i in range(len(dev_fuzz_similarity)):
    dev_fuzz_similarity[i] = dev_fuzz_similarity[i] / 10

In [509]:
train_labels = list(train_labels)
dev_labels = list(dev_labels)

for i in range(len(dev_labels)):
    try:
        dev_labels[i] = int(dev_labels[i])
    except:
        dev_labels[i] = 0
        
for i in range(len(train_labels)):
    try:
        train_labels[i] = int(train_labels[i])
    except:
        train_labels[i] = 0

In [510]:
fuzz_similarity = [[i] for i in fuzz_similarity]
dev_fuzz_similarity = [[i] for i in dev_fuzz_similarity]

In [511]:
ros = RandomOverSampler(sampling_strategy=0.4)
fuzz_dup, labels_dup = ros.fit_resample(fuzz_similarity, train_labels)

In [515]:
def build_MLP(train_features, test_features, train_targets, test_targets):
    classifier = MLPClassifier(hidden_layer_sizes=(25,15), activation='tanh', batch_size=300, n_iter_no_change = 100, solver='adam', max_iter=3000, learning_rate_init=0.008, learning_rate='adaptive')
    classifier.fit(train_features, train_targets)
    predictions = classifier.predict(test_features)
    print(predictions)
    score = f1_score(test_targets, predictions)
    accuracy = accuracy_score(test_targets, predictions)
    print(score)
    print(accuracy)
    if score == 0.0:
        print("didnt work")
    return score

In [513]:
predictions = build_MLP(fuzz_dup, dev_fuzz_similarity, labels_dup, dev_labels)

[0 0 0 ... 0 1 0]
0.7792329279700656
0.882


In [516]:
lowest_pred = 1
for i in range(0,30):
    predictions = build_MLP(fuzz_dup, dev_fuzz_similarity, labels_dup, dev_labels)
    if(predictions < lowest_pred):
        lowest_pred = predictions
print(lowest_pred)

[0 0 0 ... 0 1 0]
0.7500000000000001
0.866
[0 0 0 ... 0 1 0]
0.7492877492877492
0.868
[0 0 0 ... 0 1 0]
0.7682869277961303
0.87725
[0 0 0 ... 0 1 0]
0.7638483965014577
0.8785
[0 0 0 ... 0 1 0]
0.7805530776092774
0.877
[0 0 0 ... 0 1 0]
0.7380365071534286
0.86725
[0 0 0 ... 0 1 0]
0.7698042870456663
0.8765
[0 0 0 ... 0 1 0]
0.7583985273815002
0.86875
[0 0 0 ... 0 1 0]
0.7474747474747476
0.86875
[0 0 0 ... 0 1 0]
0.7500000000000001
0.866
[0 0 0 ... 0 1 0]
0.7595549104983066
0.87575
[0 0 0 ... 0 1 0]
0.7701465201465201
0.8745
[0 0 0 ... 0 1 0]
0.7698042870456663
0.8765
[0 0 0 ... 0 1 0]
0.7492877492877492
0.868
[0 0 0 ... 0 1 0]
0.7701465201465201
0.8745
[0 0 0 ... 0 1 0]
0.7595549104983066
0.87575
[0 0 0 ... 0 1 0]
0.7701465201465201
0.8745
[0 0 0 ... 0 1 0]
0.7492877492877492
0.868
[0 0 0 ... 0 1 0]
0.7698042870456663
0.8765
[0 0 0 ... 0 1 0]
0.7474747474747476
0.86875
[0 0 0 ... 0 1 0]
0.7380365071534286
0.86725
[0 0 0 ... 0 1 0]
0.7698042870456663
0.8765
[0 0 0 ... 0 1 0]
0.7698042870

In [240]:
df = pd.DataFrame(predictions)

In [ ]:
#fuzz_similarity_dup = []
#for i in range(len(fuzz_similarity)):
  #  if(fuzz_similarity[i] >= 8.5):
   #     fuzz_similarity

In [470]:
dev_stuff = pd.DataFrame({'similarity': dev_fuzz_similarity, 'gold_label': dev_labels, 'predicted': predictions})

In [471]:
dev_stuff[0:50]

,similarity,gold_label,predicted
0,[100],0,0
1,[45],0,0
2,[100],0,0
3,[100],0,0
4,[92],1,1
5,[67],1,0
6,[81],0,1
7,[100],0,0
8,[100],0,0
9,[100],0,0
